In [1]:
#---- to do list -----
# err_data_list 파일자동화
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

from tensorflow.python.keras.optimizer_v2.rmsprop import RMSProp
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, RepeatVector, LSTM, Input, TimeDistributed, Activation, Dropout
from keras.optimizers import SGD
np.set_printoptions(suppress=True)

#pow 낮값만 추출 test
#pow = 0인 구간 : 0~4, 21-23시
powhr_start = 5
powhr_end   = 20

shift_days  = 3
hoursteps   = powhr_end-powhr_start+1 #(16)
timesteps   = shift_days*hoursteps #hours step

data_dim    = 7
out_dim     = 1
n_model    = 10

data_dir  = 'C:/Users/VISLAB_PHY/Desktop/Workspace/Data'

season_mod = 'all_1102_f7'
date_start = '10190901'
date_end   = '30191201'

err_date_list = ['20190912',
                    '20191122',
                    '20191130',
                    '20191217',
                    '20200501',
                    '20200502',
                    '20191028',
                    '20191107',
                    '20191108',
                    '20191109',
                    '20191110',
                    '20191111',
                    '20191112',
                    '20200214',
                    '20200307',
                    '20200308',
                    '20200309',
                    '20200310',
                    '20200328',
                    '20200329',
                    '20200625',
                    '20200809']

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PowerTransformer 

transf_type = 'yeo-johnson'
#transf_type = 'box-cox'

def yeo_johnson_transform(dataset):
    
    column_trans = ColumnTransformer(
                    [('sunshine', PowerTransformer(method=transf_type, standardize=True), ['sunshine']),
                     ('humdt', PowerTransformer(method=transf_type, standardize=True), ['humdt']),
                     ('wnd_spd', PowerTransformer(method=transf_type, standardize=True), ['wnd_spd']),
                     ('visiblt', PowerTransformer(method=transf_type, standardize=True), ['visiblt']),
                     ('cloud2', PowerTransformer(method=transf_type, standardize=True), ['cloud2']),
                     ('cloud', PowerTransformer(method=transf_type, standardize=True), ['cloud']),
                     ('grd_temprt', PowerTransformer(method=transf_type, standardize=True), ['grd_temprt']),
                     ('wnd_dir', PowerTransformer(method=transf_type, standardize=True), ['wnd_dir']),
                     ('dewpnt', PowerTransformer(method=transf_type, standardize=True), ['dewpnt']),
                     ('steampressr', PowerTransformer(method=transf_type, standardize=True), ['steampressr']),
                     ('temprt', PowerTransformer(method=transf_type, standardize=True), ['temprt']),
                     ('mincloud', PowerTransformer(method=transf_type, standardize=True), ['mincloud']),
                     ('rain', PowerTransformer(method=transf_type, standardize=True), ['rain']),
                     ('pressr', PowerTransformer(method=transf_type, standardize=True), ['pressr']),
                     ('seapressr', PowerTransformer(method=transf_type, standardize=True), ['seapressr']),
                     ('snow', PowerTransformer(method=transf_type, standardize=True), ['snow'])
                    ])
    
    transformed_data = column_trans.fit_transform(dataset)
    transformed_df = pd.DataFrame(transformed_data, columns=dataset.columns)
    pd.concat([transformed_df], axis = 1)
                    
    return transformed_df

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import fnmatch
from pandas import read_csv
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PowerTransformer 
#from feature_engine import variable_transformers as vt
from scipy.stats import yeojohnson

#############################################
# 태양광 전력
#############################################
def get_pow():

    # pow 파일 load
    dir_path    = data_dir+"/pow_24/UR00000126_csv"
    file_list   = os.listdir(dir_path)
    print(len(file_list))
    hrPow  = []    

    # pow측정값 에러가 큰 일자 제거
    for filename in file_list:
        if (filename[:-4] not in err_date_list):
            if ((filename[:-4]>=date_start) & (filename<date_end)):
                filedata = pd.read_csv(dir_path+'/'+filename).values[:,0]
                hrPow.append(filedata)
                
    #낮시간 추출 (5~20시)
    pow_dataset = pd.DataFrame(hrPow)
    pow_dataset =pow_dataset.iloc[:,powhr_start:powhr_end+1]
    #pow_dataset.to_csv("C:/Users/VISLAB_PHY/Desktop/WORKSPACE/Origin/data/pow_hr.csv",mode='w',index=False)

    # 결측값 보간, reshape
    pow_dataset = pow_dataset.interpolate(method='linear')
    pow_dataset = pow_dataset.values.reshape(-1,1)
    pow_dataset = pd.DataFrame(pow_dataset)
    pow_dataset.columns = ['pow']
    pow_dataset.to_csv(data_dir+"/pow.csv",mode='w',index=False)

    
    # scale
    sc_pow = MinMaxScaler(feature_range = (0, 1))
    scaled_pow = sc_pow.fit_transform(pow_dataset.values)
    df_pow = pd.DataFrame(scaled_pow, columns=pow_dataset.columns, index=list(pow_dataset.index.values))
    
    return df_pow, sc_pow

#############################################
# 종관기상관측
#############################################
def get_weather():
    # pow 파일 load
    file_list   = os.listdir(data_dir)
    print(len(file_list))
    for filename in os.listdir(data_dir):
        if fnmatch.fnmatch(filename, 'OBS_ASOS_TIM_*.csv'):
            print(filename)

            # load csv data
            dataset = read_csv(data_dir+'/'+filename, encoding='CP949')
            dataset.drop(['지점','지점명'], axis=1, inplace=True)
            dataset.drop(['기온 QC플래그','강수량 QC플래그','풍속 QC플래그','풍향 QC플래그','습도 QC플래그'], axis=1, inplace=True)
            dataset.drop(['현지기압 QC플래그','해면기압 QC플래그','일조 QC플래그','지면온도 QC플래그'], axis=1, inplace=True)
            dataset.drop(['5cm 지중온도(°C)','10cm 지중온도(°C)','20cm 지중온도(°C)','30cm 지중온도(°C)'], axis=1, inplace=True)
            dataset.drop(['3시간신적설(cm)','일사(MJ/m2)','운형(운형약어)','지면상태(지면상태코드)','현상번호(국내식)'], axis=1, inplace=True)

            # set column name
            dataset.columns = ['ymdhms', 'temprt', 'rain', 'wnd_spd', 'wnd_dir', 'humdt','steampressr',
                               'dewpnt', 'pressr','seapressr','sunshine','snow','cloud','cloud2','mincloud','visiblt','grd_temprt']

            # prioirty sort (피어슨상관계수)
            dataset = dataset[['ymdhms','sunshine','humdt','wnd_spd','visiblt','cloud2',
                               'cloud','grd_temprt','wnd_dir','dewpnt','steampressr','temprt',
                               'mincloud','rain','pressr','seapressr','snow']]


            # set NA data (관측값 0이 누적되어 결측된 경우. 0으로 세팅)
            dataset['rain'].fillna(0, inplace=True)     #강수량
            dataset['sunshine'].fillna(0, inplace=True) #일조
            dataset['snow'].fillna(0, inplace=True)     #적설량

            #일시 패턴 변환(2019-08-20 5:00 -> 2019082005)
            dataset['ymdhms'] = dataset['ymdhms'].str[0:4]+dataset['ymdhms'].str[5:7]+dataset['ymdhms'].str[8:10]+dataset['ymdhms'].str[11:13]
            # pow측정값 중 결측값 많은 일자 제거
            dataset = dataset[(dataset['ymdhms'].str[0:8]>=date_start) & (dataset['ymdhms'].str[0:8]<date_end)]
            for err_date in err_date_list:
                idx_err = dataset[dataset['ymdhms'].str.startswith(err_date)].index
                dataset = dataset.drop(idx_err)

            #낮시간 추출 (5~20시)
            dataset = dataset[(dataset['ymdhms'].str[-2:]>=str(powhr_start).rjust(2, '0')) &(dataset['ymdhms'].str[-2:]<=str(powhr_end))]
            dataset = dataset.interpolate(method='linear')# 결측값 보간
            
            # save file (test용)
            dataset.to_csv(data_dir+"/weather.csv",mode='w',index=False)

            # normalization
            dataset.drop(['ymdhms'], axis=1, inplace=True)
            dataset = dataset.astype('float32')
            dataset = dataset.interpolate(method='linear')
            
            #YEO-JOHNSON transform
            yeo_df = yeo_johnson_transform(dataset)
            
            #insert feature (test)
            yeo_df.insert(2, 'temp_press', yeo_df['temprt']-yeo_df['steampressr'], True)
            yeo_df.insert(2, 'sunshine_humdt', abs(yeo_df['sunshine'])-(yeo_df['humdt']*(2.1)), True)#0.35
            
            sc = MinMaxScaler(feature_range = (0, 1))#scale
            scaled_weather = sc.fit_transform(yeo_df.values)
            weather = pd.DataFrame(scaled_weather, columns=yeo_df.columns, index=list(yeo_df.index.values))
            print("before : ", weather.shape)
            weather = weather.iloc[:, 0:data_dim] #feature size 조절
            print("after : ", weather.shape)
            
    return weather


#############################################
# numpy data 만들기
#############################################

df_pow, sc_pow   = get_pow()
df               = get_weather()

# pow + weather + powY
df.insert(0, 'pow', df_pow.values, True)
df = df.iloc[0:-timesteps, :]
df.insert(df.shape[1], 'pow_Y', df_pow.iloc[timesteps:, :].values, True)

df.to_csv(data_dir+"/total.csv",mode='w',index=False, encoding='CP949')

#----------------------------------------------
# time step만큼 window 움직여 dataset 생성
#----------------------------------------------
totalsize = df.shape[0]
dataX, dataY = [], []

for i in range(0, totalsize-timesteps-24+1, hoursteps):
    dataX.append(df.iloc[i:(i + timesteps),0:-1])
    dataY.append(df.iloc[i:(i + hoursteps),[0]])

print("len(dataX) : ", len(dataX), dataX[0].shape)
print("len(dataY) : ", len(dataY), dataY[0].shape)

#----------------------------------------------
#  Split train/test 
#----------------------------------------------

train_size = int(len(dataX) * 0.7)
val_size   = int(len(dataX) * 0.2)
test_size  = len(dataX) - train_size - val_size
val_idx = train_size+val_size

trainX, valX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:val_idx]), np.array(dataX[val_idx:val_idx+test_size])
trainY, valY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:val_idx]), np.array(dataY[val_idx:val_idx+test_size])

print('train X : ', trainX.shape, '\tY : ', trainY.shape)
print('val   X : ', valX.shape,   '\tY : ', valY.shape)
print('test  X : ', testX.shape,  '\tY : ', testY.shape)

np.save("npset/"+season_mod+"_trainX",trainX)
np.save("npset/"+season_mod+"_trainY",trainY)
np.save("npset/"+season_mod+"_valX",valX)
np.save("npset/"+season_mod+"_valY",valY)
np.save("npset/"+season_mod+"_testX",testX)
np.save("npset/"+season_mod+"_testY",testY)


model = Sequential()
model.add(LSTM(256, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(RepeatVector(hoursteps))
model.add(LSTM(256, return_sequences=True))
model.add(TimeDistributed(Dense(256, activation='relu')))
model.add(TimeDistributed(Dense(1)))
model.summary()

Using TensorFlow backend.


337
14
OBS_ASOS_TIM_20200929025447.csv


c:\users\vislab_phy\anaconda3\envs\py37\lib\site-packages\numpy\core\_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
c:\users\vislab_phy\anaconda3\envs\py37\lib\site-packages\numpy\core\_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
c:\users\vislab_phy\anaconda3\envs\py37\lib\site-packages\numpy\core\_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
c:\users\vislab_phy\anaconda3\envs\py37\lib\site-packages\numpy\core\_methods.py:205: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


before :  (5344, 18)
after :  (5344, 7)
len(dataX) :  327 (48, 8)
len(dataY) :  327 (16, 1)
train X :  (228, 48, 8) 	Y :  (228, 16, 1)
val   X :  (65, 48, 8) 	Y :  (65, 16, 1)
test  X :  (34, 48, 8) 	Y :  (34, 16, 1)


RuntimeError: It looks like you are trying to use a version of multi-backend Keras that does not support TensorFlow 2.0. We recommend using `tf.keras`, or alternatively, downgrading to TensorFlow 1.14.

In [2]:
n_model = 3;
for i in range(n_model):#0,5):#

    #keras.optimizers.RMSprop(lr=0.005, rho=0.9, epsilon=None, decay=0.0)
    model.compile(loss='mean_squared_error', 
                  optimizer=RMSProp()
                  #optimizer=RMSProp(learning_rate=0.001)
                  #optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True), 
                  #metrics=['acc'])
                 )

    hist = model.fit(trainX, 
                     trainY, 
                     epochs=300, 
                     batch_size=64,
                     validation_data=(valX, valY))


    results = model.evaluate(testX, testY)
    #model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=0)

    model.save('model/model_'+season_mod+'_'+str(i)+'.h5')# # of feature=3,5,7,9,?,12,14,16,18

Train on 228 samples, validate on 65 samples
Epoch 1/300
228/228 [==============================] - 1s 4ms/step - loss: 0.1358 - val_loss: 0.0410
Epoch 2/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0382 - val_loss: 0.0359
Epoch 3/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0360 - val_loss: 0.0398
Epoch 4/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0375 - val_loss: 0.0331
Epoch 5/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0329 - val_loss: 0.0452
Epoch 6/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0412 - val_loss: 0.0315
Epoch 7/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0290 - val_loss: 0.0953
Epoch 8/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0513 - val_loss: 0.0307
Epoch 9/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0346 - val_loss: 0.0354
Epoch 10/300
228/228 [========================

Epoch 81/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0038 - val_loss: 0.0067
Epoch 82/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0089 - val_loss: 0.0174
Epoch 83/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0101 - val_loss: 0.0071
Epoch 84/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0057 - val_loss: 0.0047
Epoch 85/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0039 - val_loss: 0.0073
Epoch 86/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0052 - val_loss: 0.0062
Epoch 87/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0058 - val_loss: 0.0061
Epoch 88/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0081 - val_loss: 0.0106
Epoch 89/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0069 - val_loss: 0.0043
Epoch 90/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0040

228/228 [==============================] - 0s 1ms/step - loss: 0.0026 - val_loss: 0.0048
Epoch 240/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0034 - val_loss: 0.0038
Epoch 241/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0022 - val_loss: 0.0031
Epoch 242/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0033
Epoch 243/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0024 - val_loss: 0.0027
Epoch 244/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0019 - val_loss: 0.0029
Epoch 245/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0018 - val_loss: 0.0066
Epoch 246/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0042 - val_loss: 0.0070
Epoch 247/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0046 - val_loss: 0.0040
Epoch 248/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0017 - v

c:\users\vislab_phy\anaconda3\envs\py37\lib\site-packages\keras\engine\saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


Train on 228 samples, validate on 65 samples
Epoch 1/300
228/228 [==============================] - 1s 3ms/step - loss: 0.0080 - val_loss: 0.0092
Epoch 2/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0044 - val_loss: 0.0031
Epoch 3/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0017 - val_loss: 0.0028
Epoch 4/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0013 - val_loss: 0.0043
Epoch 5/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0024 - val_loss: 0.0047
Epoch 6/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0025 - val_loss: 0.0032
Epoch 7/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0035
Epoch 8/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0021 - val_loss: 0.0036
Epoch 9/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0018 - val_loss: 0.0030
Epoch 10/300
228/228 [========================

...

c:\users\vislab_phy\anaconda3\envs\py37\lib\site-packages\keras\engine\saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


Train on 228 samples, validate on 65 samples
Epoch 1/300
228/228 [==============================] - 1s 3ms/step - loss: 0.0054 - val_loss: 0.0041
Epoch 2/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 0.0036
Epoch 3/300
228/228 [==============================] - 0s 1ms/step - loss: 8.1129e-04 - val_loss: 0.0035
Epoch 4/300
228/228 [==============================] - 0s 1ms/step - loss: 6.8310e-04 - val_loss: 0.0033
Epoch 5/300
228/228 [==============================] - 0s 1ms/step - loss: 8.9788e-04 - val_loss: 0.0032
Epoch 6/300
228/228 [==============================] - 0s 1ms/step - loss: 6.9981e-04 - val_loss: 0.0033
Epoch 7/300
228/228 [==============================] - 0s 1ms/step - loss: 5.0885e-04 - val_loss: 0.0034
Epoch 8/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 0.0028
Epoch 9/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 0.0034
Epoch 10/300
228/228 [====

228/228 [==============================] - 0s 1ms/step - loss: 7.7576e-04 - val_loss: 0.0053
Epoch 80/300
228/228 [==============================] - 0s 1ms/step - loss: 8.3915e-04 - val_loss: 0.0054
Epoch 81/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 0.0044
Epoch 82/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0010 - val_loss: 0.0032
Epoch 83/300
228/228 [==============================] - 0s 1ms/step - loss: 5.8054e-04 - val_loss: 0.0032
Epoch 84/300
228/228 [==============================] - 0s 1ms/step - loss: 9.3520e-04 - val_loss: 0.0026
Epoch 85/300
228/228 [==============================] - 0s 1ms/step - loss: 0.0010 - val_loss: 0.0034
Epoch 86/300
228/228 [==============================] - 0s 1ms/step - loss: 6.2471e-04 - val_loss: 0.0035
Epoch 87/300
228/228 [==============================] - 0s 1ms/step - loss: 6.0643e-04 - val_loss: 0.0055
Epoch 88/300
228/228 [==============================] - 0s 1ms/step - l

Epoch 233/300
228/228 [==============================] - 0s 1ms/step - loss: 7.2640e-04 - val_loss: 0.0031
Epoch 234/300
228/228 [==============================] - 0s 1ms/step - loss: 9.0085e-04 - val_loss: 0.0037
Epoch 235/300
228/228 [==============================] - 0s 1ms/step - loss: 5.2751e-04 - val_loss: 0.0043
Epoch 236/300
228/228 [==============================] - 0s 1ms/step - loss: 5.7499e-04 - val_loss: 0.0042
Epoch 237/300
228/228 [==============================] - 0s 1ms/step - loss: 4.3699e-04 - val_loss: 0.0053
Epoch 238/300
228/228 [==============================] - 0s 1ms/step - loss: 7.1956e-04 - val_loss: 0.0043
Epoch 239/300
228/228 [==============================] - 0s 1ms/step - loss: 9.5061e-04 - val_loss: 0.0039
Epoch 240/300
228/228 [==============================] - 0s 1ms/step - loss: 3.6512e-04 - val_loss: 0.0036
Epoch 241/300
228/228 [==============================] - 0s 1ms/step - loss: 4.6767e-04 - val_loss: 0.0034
Epoch 242/300
228/228 [==============

c:\users\vislab_phy\anaconda3\envs\py37\lib\site-packages\keras\engine\saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


In [8]:

from keras.models import load_model

#get test data
X_test = np.load("npset/"+season_mod+"_testX.npy")
y_test = np.load("npset/"+season_mod+"_testY.npy")

#get pow scale form
#powdata, scaler = libs_yeon.get_pow()

print("X_test : ", X_test.shape)
print("y_test : ", y_test.shape)

n_dataset   = y_test.shape[0]
acc_list    = []
acc_model   = []
model       = []

for i in range(n_model):
    model.append(load_model('model/model_'+season_mod+'_'+str(i)+'.h5'))
    acc_model.append(0)
    
print("[ dataset ]")
for i in range(n_dataset):
    #if(i in [0,5,13,14,24,25]): continue;
    y = sc_pow.inverse_transform(y_test[i:i+1,:,0])

    for m in range(n_model):
        #print("(model",m+1,")\t",end="")

        pred = model[m].predict([X_test[i:i+1]])
        pred[pred<0] = 0
        pred = pred[:,:,0]
        pred = sc_pow.inverse_transform(pred)
        pred = np.sum(pred)

        target      = round(np.sum(y), 2)
        error       = round(np.abs(target-pred), 2)
        error_rate  = np.min([round(error/target, 2),1])
        acc_rate    = round((1.0-error_rate)*100, 2)
        acc_list.append(acc_rate)
        acc_model[m] += acc_rate
                 
        #print("   pred: ",pred," | target: ",target," | error: ",error," | err rate: ",error_rate," | acc: ",acc_rate,sep="")
        
    #print("acc rate: ",np.mean(acc_list[-n_model:]),sep='')
    if(i%5==0): print(" ")
    print(round(np.mean(acc_list[-n_model:]), 2), " / ",sep='', end='')
print("\n----------------------------------------------")
print("mean(acc rate): ",np.mean(acc_list),sep='')
print("----------------------------------------------")
print("[ model ]")
for i in range(n_model):
    acc_model[i] = round(acc_model[i]/(n_dataset),2)
    print(acc_model[i])    

X_test :  (34, 48, 8)
y_test :  (34, 16, 1)


c:\users\vislab_phy\anaconda3\envs\py37\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


[ dataset ]
 
16.0 / 71.33 / 79.67 / 78.67 / 30.33 /  
53.67 / 75.0 / 40.33 / 86.33 / 78.67 /  
55.67 / 61.0 / 82.33 / 39.0 / 40.67 /  
77.0 / 28.33 / 77.33 / 78.0 / 48.0 /  
21.67 / 59.33 / 75.0 / 43.0 / 29.0 /  
20.33 / 83.67 / 88.67 / 58.67 / 53.33 /  
25.0 / 63.0 / 70.33 / 67.33 / 
----------------------------------------------
mean(acc rate): 57.51960784313726
----------------------------------------------
[ model ]
55.03
62.18
55.35
